In [1]:
from vizro_ai import VizroAI
from langchain_anthropic import ChatAnthropic
import os
import vizro.plotly.express as px
from vizro_ai.plot._response_models import ChartPlanStatic
from langchain_ollama import ChatOllama



ModuleNotFoundError: No module named 'vizro_ai.plot._utils'

In [ ]:
df = px.data.gapminder()

## Model

In [ ]:
# llm = ChatAnthropic(
#         model="claude-3-5-sonnet-20240620",
#         api_key = os.environ.get("ANTHROPIC_API_KEY"),
#         base_url= os.environ.get("ANTHROPIC_API_BASE")
#     )

# llm = ChatOllama(
#     model="llama3.1",
#     temperature=0,
#     # other params...
# )

# llm = None
llm = "gpt-4o-mini-2024-07-18"

In [ ]:
vizro_ai = VizroAI(model=llm)

## Query

In [2]:
# query = "plot a bubble chart to shows the changes in life expectancy gdp per capita  over time. Animate the chart by year"
# query = "describe the composition of gdp in continent,and horizontal line for avg gdp"
# query = "show me the geo distribution of life expectancy and set year as animation "
query = "what are the top 10 countries with highest gdp growth percentage wise between 2002 and 2007. Show them as a bar chart. Color the top three countries in red"

## VizroAI

### Refactored version

In [3]:
res2 = vizro_ai.plot(df=df, user_input=query, return_elements=True)

NameError: name 'vizro_ai' is not defined

In [7]:
res2.get_fig_object(data_frame=df, vizro=True)

KeyError: 'custom_chart'

In [9]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716


In [10]:
print(res2.code)

import pandas as pd
import plotly.graph_objects as go


def custom_chart(data_frame):
    # Filter data for the years 2002 and 2007
    data_2002 = data_frame[data_frame["year"] == 2002]
    data_2007 = data_frame[data_frame["year"] == 2007]

    # Merge the two dataframes on country
    merged_data = pd.merge(
        data_2002[["country", "gdpPercap"]],
        data_2007[["country", "gdpPercap"]],
        on="country",
        suffixes=("_2002", "_2007"),
    )

    # Calculate GDP growth percentage
    merged_data["gdp_growth"] = (
        (merged_data["gdpPercap_2007"] - merged_data["gdpPercap_2002"])
        / merged_data["gdpPercap_2002"]
    ) * 100

    # Get top 10 countries by GDP growth
    top_countries = merged_data.nlargest(10, "gdp_growth")

    # Create bar chart
    fig = go.Figure()
    colors = ["red" if i < 3 else "blue" for i in range(len(top_countries))]
    fig.add_trace(
        go.Bar(
            x=top_countries["country"],
            y=top_countries["gdp_gro

In [11]:
res = vizro_ai.plot(df=df, user_input=query, explain=True, return_elements=True)
res.figure

/Users/Maximilian_Schulz/Library/Application Support/hatch/env/virtual/vizro-ai/EdwJfxPm/vizro-ai/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning:

The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.



<h4>Insights:</h4>

The code snippet provided is designed to analyze and visualize the GDP growth of countries between the years 2002 and 2007. It calculates the percentage growth in GDP per capita for each country over this period and identifies the top 10 countries with the highest growth rates. The visualization highlights the top three countries in red, making it easy to identify the leaders in economic growth during this timeframe. This analysis can provide insights into which countries experienced significant economic improvements, potentially guiding investment decisions or policy considerations.
<br><br><h4>Code:</h4>

1. The code imports necessary libraries: 'capture' from 'vizro.models.types' and 'plotly.express' as 'px', along with 'pandas' as 'pd'.
2. A function 'custom_chart' is defined, which takes an optional parameter 'data_frame'.
3. Inside the function, a copy of the input DataFrame is created to avoid modifying the original data.
4. The DataFrame is filtered to include only the years 2002 and 2007.
5. A pivot table is created to reshape the data, with countries as rows and years as columns, focusing on GDP per capita.
6. A new column 'gdp_growth' is calculated to determine the percentage change in GDP per capita from 2002 to 2007.
7. The top 10 countries with the highest GDP growth are selected and prepared for visualization.
8. A bar chart is created using Plotly Express and Vizro, with countries on the x-axis and GDP growth percentages on the y-axis.
9. The color of the bars is set to a continuous scale from red to blue, with the top three countries highlighted in red.
10. The layout of the chart is updated for better aesthetics, including axis titles and legend visibility.
11. Finally, the function returns the generated figure.
<br>**This customized chart can be directly used in a Vizro dashboard.** 
Click [custom chart docs](https://vizro.readthedocs.io/en/stable/pages/user_guides/custom_charts/) for more information.
```
from vizro.models.types import capture
import vizro.plotly.express as px
import pandas as pd

@capture('graph')
def custom_chart(data_frame=None):
    df_copy = data_frame.copy()
    df_growth = df_copy[df_copy['year'].isin([2002, 2007])]
    df_growth = df_growth.pivot_table(index='country', columns='year', values='gdpPercap').reset_index()
    df_growth['gdp_growth'] = (df_growth[2007] - df_growth[2002]) / df_growth[2002] * 100
    df = df_growth.nlargest(10, 'gdp_growth')[['country', 'gdp_growth']].set_index('country').reset_index()

    # Create a bar chart for the top 10 countries with highest GDP growth
    fig = px.bar(df,
                 x='country',
                 y='gdp_growth',
                 title='Top 10 Countries with Highest GDP Growth (2002-2007)',
                 labels={'gdp_growth': 'GDP Growth Percentage (%)', 'country': 'Country'},
                 color='gdp_growth',
                 color_continuous_scale=['red', 'blue'],
                 range_color=[df['gdp_growth'].min(), df['gdp_growth'].max()])

    # Highlight the top 3 countries in red
    for i in range(min(3, len(fig.data))):
        fig.data[i].marker.color = 'red'

    # Update layout for better aesthetics
    fig.update_layout(xaxis_title='Country',
                      yaxis_title='GDP Growth Percentage (%)',
                      showlegend=False)

    return fig

fig = custom_chart(data_frame=df)
```

In [12]:
assert False

AssertionError: 

In [ ]:
print(res2.code_vizro)

In [ ]:
print(res2.code_explanation)

In [ ]:
print(res2.chart_insights)

In [ ]:
# Underlying method for code properties
print(res2._get_complete_code(chart_name="other_figure",vizro=False))

In [ ]:
# Run chart with different data
res2.get_fig_object(data_frame=df.sample(200))

In [ ]:
# No code execution
res22 = vizro_ai.plot2(df=df, user_input=query, validate_code=False, return_elements=True)
print(res22.code_vizro)

In [ ]:
res2.code

### Old version

In [ ]:
res = vizro_ai.plot(df=df, user_input=query, explain=True, return_elements=True)
res.figure

/Users/Maximilian_Schulz/Library/Application Support/hatch/env/virtual/vizro-ai/EdwJfxPm/vizro-ai/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning:

The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.



<h4>Insights:</h4>

The bar chart visualizes the top 10 countries with the highest GDP growth rates, providing insights into economic performance. High GDP growth indicates a robust economy, attracting investments and improving living standards. Countries with significant growth may be emerging markets or recovering from economic downturns. This data can guide investors and policymakers in decision-making, highlighting regions with potential for economic expansion.
<br><br><h4>Code:</h4>

1. Import necessary libraries: vizro.models.types for capturing the graph and vizro.plotly.express for plotting. 2. Define a function 'custom_chart' that takes a DataFrame as input. 3. Create a copy of the input DataFrame to avoid modifying the original data. 4. Use 'nlargest' to find the top 10 countries with the highest GDP growth from the 'gdp_growth' column. 5. Reset the index of the resulting DataFrame for cleaner visualization. 6. Create a bar chart using Plotly Express and Vizro, setting the x-axis to 'country' and y-axis to 'gdp_growth'. 7. Customize the chart with titles, labels, and color scales for better readability. 8. Update the layout for improved aesthetics, including axis titles and tick angle. 9. Return the generated figure for display.
<br>**This customized chart can be directly used in a Vizro dashboard.** 
Click [custom chart docs](https://vizro.readthedocs.io/en/stable/pages/user_guides/custom_charts/) for more information.
```
from vizro.models.types import capture
import vizro.plotly.express as px
import pandas as pd

@capture('graph')
def custom_chart(data_frame=None):
    df_copy = data_frame.copy()
    top_gdp_growth = df_copy[['country', 'gdp_growth']].nlargest(10, 'gdp_growth')
    df = top_gdp_growth.reset_index(drop=True, inplace=True)

    # Create a bar chart for the top 10 countries with the highest GDP growth
    fig = px.bar(
        top_gdp_growth,
        x='country',
        y='gdp_growth',
        title='Top 10 Countries with Highest GDP Growth',
        labels={'gdp_growth': 'GDP Growth (%)', 'country': 'Country'},
        color='gdp_growth',
        color_continuous_scale=px.colors.sequential.Viridis
    )

    # Update layout for better readability
    fig.update_layout(
        xaxis_title='Country',
        yaxis_title='GDP Growth (%)',
        xaxis_tickangle=-45,
        template='plotly_white'
    )

    return fig

fig = custom_chart(data_frame=df)
```

# New version with Anthropic

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True))

import os
# not all these 3 are needed. I just forgot which one is correct
print(os.environ.get("ANTHROPIC_API_URL"))
print(os.environ.get("ANTHROPIC_BASE_URL"))
print(os.environ.get("ANTHROPIC_API_BASE"))

from vizro_ai import VizroAI
vizro_ai = VizroAI(model="claude-3-sonnet-20240229")

vizro_ai.model

import vizro.plotly.express as px
query = "plot a bubble chart to shows the changes in life expectancy gdp per capita  over time. Animate the chart by year"
df = px.data.gapminder()

res2 = vizro_ai.plot2(df=df, user_input=query, return_elements=True)
res2.get_fig_object(data_frame=df)